In [1]:
#packages
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [2]:
df = pd.read_csv('GE.csv') #data can be downloaded from here https://finance.yahoo.com/quote/GE/history?p=GE
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-05-21,6.42,6.55,6.33,6.48,6.448930,98821100
1,2020-05-22,6.48,6.48,6.27,6.41,6.379265,69586200
2,2020-05-26,6.79,6.94,6.79,6.80,6.767395,129510700
3,2020-05-27,7.25,7.43,7.09,7.29,7.255046,147750900
4,2020-05-28,7.49,7.49,6.74,6.78,6.747491,138714800


In [3]:
train_dates = pd.to_datetime(df['Date'])
train_dates

0     2020-05-21
1     2020-05-22
2     2020-05-26
3     2020-05-27
4     2020-05-28
         ...    
248   2021-05-17
249   2021-05-18
250   2021-05-19
251   2021-05-20
252   2021-05-21
Name: Date, Length: 253, dtype: datetime64[ns]

In [4]:
df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

In [5]:
df_for_training = df[list(df)[1:5]].astype(float)
df_for_training

,Open,High,Low,Close
0,6.42,6.550,6.33,6.480
1,6.48,6.480,6.27,6.410
2,6.79,6.940,6.79,6.800
3,7.25,7.430,7.09,7.290
4,7.49,7.490,6.74,6.780
...,...,...,...,...
248,13.23,13.340,13.12,13.150
249,13.18,13.270,12.97,12.970
250,12.90,13.100,12.79,13.090
251,13.09,13.110,12.92,13.060


In [6]:
# scaling the data for efficient computtation
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [7]:
#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 
#In this example, the n_features is 2. We will make timesteps = 3. 
#With this, the resultant n_samples is 5 (as the input data has 9 rows).
trainX = []
trainY = []

n_future = 1   # Number of days we want to predict into the future
n_past = 14     # Number of past days we want to use to predict the future

for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (239, 14, 4).
trainY shape == (239, 1).


In [8]:
trainX

array([[[-1.1265945 , -1.12810962, -1.10982967, -1.09989985],
        [-1.10493279, -1.1531506 , -1.13174417, -1.12511127],
        [-0.99301395, -0.98859558, -0.94181846, -0.98464764],
        ...,
        [-0.47674316, -0.40549846, -0.45969934, -0.3867768 ],
        [-0.48757402, -0.53428064, -0.53640011, -0.54524859],
        [-0.56700029, -0.61655815, -0.67519198, -0.69291548]],

       [[-1.10493279, -1.1531506 , -1.13174417, -1.12511127],
        [-0.99301395, -0.98859558, -0.94181846, -0.98464764],
        [-0.82694083, -0.81330872, -0.83224593, -0.80816769],
        ...,
        [-0.48757402, -0.53428064, -0.53640011, -0.54524859],
        [-0.56700029, -0.61655815, -0.67519198, -0.69291548],
        [-0.89553625, -0.84192698, -0.92355637, -0.93062316]],

       [[-0.99301395, -0.98859558, -0.94181846, -0.98464764],
        [-0.82694083, -0.81330872, -0.83224593, -0.80816769],
        [-0.74029398, -0.79184502, -0.96008055, -0.9918509 ],
        ...,
        [-0.56700029, -0.61

In [9]:
trainY

array([[-0.89553625],
       [-0.79444826],
       [-0.93524938],
       [-0.65364714],
       [-0.7366837 ],
       [-0.87026425],
       [-0.76556598],
       [-0.85582311],
       [-0.88470539],
       [-0.95330081],
       [-1.09771222],
       [-1.0543888 ],
       [-1.07244022],
       [-1.02189623],
       [-0.94246995],
       [-0.96774195],
       [-0.92441853],
       [-0.9569111 ],
       [-0.98218309],
       [-0.97857281],
       [-1.08327108],
       [-1.0038448 ],
       [-1.03994766],
       [-0.88470539],
       [-0.89553625],
       [-0.89553625],
       [-0.91358767],
       [-0.93885967],
       [-0.92080824],
       [-0.89553625],
       [-0.90997739],
       [-0.97496252],
       [-1.02550652],
       [-0.92080824],
       [-1.09771222],
       [-1.18796935],
       [-1.22768249],
       [-1.24212363],
       [-1.21685163],
       [-1.15908707],
       [-1.17352821],
       [-1.12298422],
       [-0.96413167],
       [-0.96413167],
       [-1.06521965],
       [-1

In [10]:
#train-test split
trainx, testx = train_test_split(trainX)
trainy, testy = train_test_split(trainY)

In [11]:
print(trainx.shape,trainy.shape,testx.shape,testy.shape)

(179, 14, 4) (179, 1) (60, 14, 4) (60, 1)


In [12]:
#building the LSTM model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainx.shape[1], trainx.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainy.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()


# fit model
history = model.fit(trainx, trainy, epochs=500, batch_size=16, validation_split=0.1, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 14, 64)            17664     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 30,113
Trainable params: 30,113
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
11/11 [==============================] - 35s 207ms/step - loss: 1.1424 - val_loss: 0.9218
Epoch 2/500
11/11 [==============================] - 0s 19ms/step - loss: 1.0606 - val_loss: 0.9056
Epoch 3/500
11/11 [=============================

11/11 [==============================] - 0s 20ms/step - loss: 0.9573 - val_loss: 0.8751
Epoch 72/500
11/11 [==============================] - 0s 20ms/step - loss: 0.9654 - val_loss: 0.8739
Epoch 73/500
11/11 [==============================] - 0s 18ms/step - loss: 0.9680 - val_loss: 0.8754
Epoch 74/500
11/11 [==============================] - 0s 23ms/step - loss: 0.9923 - val_loss: 0.8750
Epoch 75/500
11/11 [==============================] - 0s 25ms/step - loss: 0.9758 - val_loss: 0.8724
Epoch 76/500
11/11 [==============================] - 0s 26ms/step - loss: 0.9490 - val_loss: 0.8711
Epoch 77/500
11/11 [==============================] - ETA: 0s - loss: 1.071 - ETA: 0s - loss: 1.055 - 0s 26ms/step - loss: 1.0486 - val_loss: 0.8817
Epoch 78/500
11/11 [==============================] - 0s 21ms/step - loss: 0.9545 - val_loss: 0.8806
Epoch 79/500
11/11 [==============================] - 0s 19ms/step - loss: 0.9890 - val_loss: 0.8814
Epoch 80/500
11/11 [==============================] - 0s

11/11 [==============================] - 0s 23ms/step - loss: 1.0233 - val_loss: 0.8671
Epoch 148/500
11/11 [==============================] - ETA: 0s - loss: 1.082 - 0s 23ms/step - loss: 1.0685 - val_loss: 0.8635
Epoch 149/500
11/11 [==============================] - 0s 23ms/step - loss: 1.0206 - val_loss: 0.8594
Epoch 150/500
11/11 [==============================] - 0s 20ms/step - loss: 0.9225 - val_loss: 0.8581
Epoch 151/500
11/11 [==============================] - 0s 18ms/step - loss: 1.0396 - val_loss: 0.8615
Epoch 152/500
11/11 [==============================] - 0s 21ms/step - loss: 0.9711 - val_loss: 0.8634
Epoch 153/500
11/11 [==============================] - 0s 26ms/step - loss: 0.9803 - val_loss: 0.8605
Epoch 154/500
11/11 [==============================] - 0s 25ms/step - loss: 0.9496 - val_loss: 0.8563
Epoch 155/500
11/11 [==============================] - 0s 28ms/step - loss: 0.9477 - val_loss: 0.8630
Epoch 156/500
11/11 [==============================] - 0s 28ms/step - lo

Epoch 225/500
11/11 [==============================] - 0s 19ms/step - loss: 1.0040 - val_loss: 0.8302
Epoch 226/500
11/11 [==============================] - 0s 17ms/step - loss: 0.9389 - val_loss: 0.8284
Epoch 227/500
11/11 [==============================] - 0s 18ms/step - loss: 0.9409 - val_loss: 0.8236
Epoch 228/500
11/11 [==============================] - 0s 17ms/step - loss: 1.0434 - val_loss: 0.8363
Epoch 229/500
11/11 [==============================] - 0s 18ms/step - loss: 1.0237 - val_loss: 0.8386
Epoch 230/500
11/11 [==============================] - ETA: 0s - loss: 0.885 - ETA: 0s - loss: 0.909 - 0s 25ms/step - loss: 0.9234 - val_loss: 0.8351
Epoch 231/500
11/11 [==============================] - 0s 21ms/step - loss: 0.9671 - val_loss: 0.8534
Epoch 232/500
11/11 [==============================] - 0s 20ms/step - loss: 0.9878 - val_loss: 0.8500
Epoch 233/500
11/11 [==============================] - 0s 20ms/step - loss: 0.9244 - val_loss: 0.8417
Epoch 234/500
11/11 [=============

11/11 [==============================] - 0s 23ms/step - loss: 0.9596 - val_loss: 0.8473
Epoch 303/500
11/11 [==============================] - 0s 18ms/step - loss: 0.9296 - val_loss: 0.9222
Epoch 304/500
11/11 [==============================] - 0s 23ms/step - loss: 0.9169 - val_loss: 0.8313 ETA: 0s - loss: 0.914
Epoch 305/500
11/11 [==============================] - 0s 21ms/step - loss: 0.9549 - val_loss: 0.8899
Epoch 306/500
11/11 [==============================] - 0s 21ms/step - loss: 0.9575 - val_loss: 0.8235
Epoch 307/500
11/11 [==============================] - 0s 18ms/step - loss: 0.8787 - val_loss: 0.7960
Epoch 308/500
11/11 [==============================] - 0s 17ms/step - loss: 0.9040 - val_loss: 0.8436
Epoch 309/500
11/11 [==============================] - 0s 18ms/step - loss: 0.8793 - val_loss: 0.8383
Epoch 310/500
11/11 [==============================] - 0s 20ms/step - loss: 0.9729 - val_loss: 0.9121
Epoch 311/500
11/11 [==============================] - 0s 22ms/step - loss

11/11 [==============================] - 0s 20ms/step - loss: 0.9415 - val_loss: 0.8463
Epoch 381/500
11/11 [==============================] - 0s 19ms/step - loss: 0.9232 - val_loss: 0.8619- ETA: 0s - loss: 0.926
Epoch 382/500
11/11 [==============================] - 0s 18ms/step - loss: 0.8934 - val_loss: 0.8598
Epoch 383/500
11/11 [==============================] - ETA: 0s - loss: 1.110 - ETA: 0s - loss: 1.045 - ETA: 0s - loss: 1.008 - 0s 24ms/step - loss: 0.9909 - val_loss: 0.8225
Epoch 384/500
11/11 [==============================] - ETA: 0s - loss: 0.945 - ETA: 0s - loss: 0.931 - 0s 24ms/step - loss: 0.9249 - val_loss: 0.8567
Epoch 385/500
11/11 [==============================] - ETA: 0s - loss: 0.9041- ETA: 0s - loss: 0.90 - 0s 25ms/step - loss: 0.9031 - val_loss: 0.8492
Epoch 386/500
11/11 [==============================] - 0s 28ms/step - loss: 0.8744 - val_loss: 0.8315- ETA: 0s - loss: 0.870
Epoch 387/500
11/11 [==============================] - 0s 22ms/step - loss: 0.8480 - va

11/11 [==============================] - 0s 14ms/step - loss: 0.8882 - val_loss: 0.8150
Epoch 457/500
11/11 [==============================] - 0s 15ms/step - loss: 0.8728 - val_loss: 0.9075
Epoch 458/500
11/11 [==============================] - 0s 14ms/step - loss: 0.8685 - val_loss: 0.9559
Epoch 459/500
11/11 [==============================] - 0s 15ms/step - loss: 0.8511 - val_loss: 0.9525
Epoch 460/500
11/11 [==============================] - 0s 16ms/step - loss: 0.8223 - val_loss: 0.8673
Epoch 461/500
11/11 [==============================] - 0s 14ms/step - loss: 0.8787 - val_loss: 0.9361
Epoch 462/500
11/11 [==============================] - 0s 15ms/step - loss: 0.9304 - val_loss: 0.9778
Epoch 463/500
11/11 [==============================] - 0s 16ms/step - loss: 0.8216 - val_loss: 0.9085
Epoch 464/500
11/11 [==============================] - 0s 15ms/step - loss: 0.8704 - val_loss: 0.9300
Epoch 465/500
11/11 [==============================] - 0s 17ms/step - loss: 0.7845 - val_loss: 1

In [13]:
#testing the model
x_input = testx
temp_input=list(x_input)
lst_output=[]
i=0
while(i<10):
    
    if(len(temp_input)>14):
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        #print(x_input)
        #x_input = x_input.reshape((1, n_steps, n_features))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        scaler.inverse_transform(yhat)
        print("{} day output {}".format(i,yhat))
        temp_input.append(yhat[0][0])
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.append(yhat[0][0])
        i=i+1
    else:
        #x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.append(yhat[0][0])
        lst_output.append(yhat[0][0])
        i=i+1
    print('\n')
    

print(lst_output)
lstm = lst_output

0 day input [[[ 0.44748986  0.5067087   0.50088648  0.45960663]
  [ 0.33557102  0.41727663  0.36209461  0.4452001 ]
  [ 0.42221786  0.37792651  0.40957604  0.38757399]
  ...
  [ 0.46554129  0.68557285  0.53375824  0.65769636]
  [ 0.72909211  0.66768643  0.68350736  0.62528168]
  [ 0.63161441  0.62118176  0.65428802  0.6504931 ]]

 [[ 0.47637214  0.4637813   0.46436231  0.43439521]
  [ 0.48359271  0.58898621  0.53741066  0.61447678]
  [ 0.61356298  0.57109979  0.62141626  0.60007026]
  ...
  [ 0.8301801   0.80362319  0.87343307  0.83777794]
  [ 0.8301801   0.77142764  0.78212264  0.75494042]
  [ 0.78685667  0.83224145  0.84421373  0.89540405]]

 [[ 1.37533317  1.42249314  1.42860054  1.39603084]
  [ 1.42948744  1.39745216  1.45781989  1.44645368]
  [ 1.47642115  1.5369662   1.52721582  1.45365694]
  ...
  [ 1.30673775  1.30086552  1.35189978  1.35641289]
  [ 1.36089202  1.38672031  1.43590538  1.37442105]
  [ 1.3175686   1.2829791   1.30441835  1.28077863]]

 ...

 [[ 0.02869677  0.0201

ValueError: non-broadcastable output operand with shape (59,1) doesn't match the broadcast shape (59,4)